In [9]:
import sys
sys.path.append("../../underactuated")

In [10]:
import numpy as np

from pendulum_swingup_fvi import convex_sampling_hjb_lower_bound
from polynomial_integration_fvi import pendulum_setup
from pydrake.examples.pendulum import (PendulumPlant)
from pydrake.all import (DiagramBuilder, Simulator, WrapToSystem, LeafSystem,
                         BasicVector)
from underactuated.pendulum import PendulumVisualizer
from underactuated.jupyter import AdvanceToAndVisualize

In [11]:
class Controller(LeafSystem):
    def __init__(self, u_star, z, plant, params_dict, J_star):
        LeafSystem.__init__(self)
        self.plant = plant
        self.context = plant.CreateDefaultContext()
        self.x_dim = 2
        self.u_dim = 1
        self.x2z = params_dict["x2z"]
        self.u_star = u_star
        self.J_star = J_star
        self.z = z

        self.state_input_port = self.DeclareVectorInputPort(
            "state", BasicVector(self.x_dim))

        self.policy_output_port = self.DeclareVectorOutputPort(
            "policy", BasicVector(self.u_dim), self.CalculateController)

    def CalculateController(self, context, output):
        state = self.state_input_port.Eval(context)
        state[0] = state[0] + np.pi
        z_value = self.x2z(state)
        y = output.get_mutable_value()
        y[:]  = u_star[0].Evaluate({z[0]: z_value[0], z[1]: z_value[1], z[2]: z_value[2]})

In [12]:
def simulate(u_star, z, params_dict):
    # Animate the resulting policy.
    builder = DiagramBuilder()
    pendulum = builder.AddSystem(PendulumPlant())

    wrap = builder.AddSystem(WrapToSystem(2))
    wrap.set_interval(0, 0, 2*np.pi)
    builder.Connect(pendulum.get_output_port(0), wrap.get_input_port(0))
    vi_policy = Controller(u_star, z, pendulum, params_dict,J_star)
    builder.AddSystem(vi_policy)
    builder.Connect(wrap.get_output_port(0), vi_policy.get_input_port(0))
    builder.Connect(vi_policy.get_output_port(0),
                    pendulum.get_input_port(0))

    visualizer = builder.AddSystem(
        PendulumVisualizer(show=False))
    builder.Connect(pendulum.get_output_port(0),
                    visualizer.get_input_port(0))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.get_mutable_context().SetContinuousState([0.1, 0])

    AdvanceToAndVisualize(simulator, visualizer, 5)

In [13]:
# params_dict = pendulum_setup()
# J_star, u_star, z = convex_sampling_hjb_lower_bound(4, params_dict,
#                                                     11, "integrate_ring")
#

In [14]:
# simulate(u_star, z, params_dict)

In [15]:
J_star2, u_star2, z2 = convex_sampling_hjb_lower_bound(2, params_dict, 6,
                                                     "integrate_ring")

Objective:  integrate_ring
Mesh x0 No. 0
Mesh x0 No. 1
Mesh x0 No. 2
Mesh x0 No. 3
Mesh x0 No. 4
Mesh x0 No. 5
Time for adding constraints:  0.01376962661743164
========== Solving ====================
 con: 2e-04    var: 0e+00    cones: 1e-11  
  Dual.    obj: -1.7429151338e+03   nrm: 2e+02    Viol.  con: 7e-11    var: 7e-06    cones: 3e-17  
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 485             
  Cones                  : 121             
  Scalar variables       : 373             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 3
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00  

In [16]:
simulate(u_star2, z2, params_dict)


   -1.817292385e+03  -1.817982048e+03  1.9e-05  0.01  
15  2.4e-04  1.1e-04  5.7e-04  8.17e-01   -1.816912002e+03  -1.817133530e+03  5.7e-06  0.01  
16  1.2e-04  5.6e-05  2.8e-04  4.67e-01   -1.816404847e+03  -1.816550854e+03  2.8e-06  0.01  
17  3.8e-05  1.8e-05  5.7e-05  7.63e-01   -1.815847988e+03  -1.815898934e+03  9.0e-07  0.01  
18  1.9e-05  8.6e-06  2.8e-05  4.05e-01   -1.815542651e+03  -1.815575146e+03  4.3e-07  0.01  
19  5.9e-06  2.7e-06  5.8e-06  7.36e-01   -1.815157554e+03  -1.815168445e+03  1.4e-07  0.01  
20  2.7e-06  1.2e-06  2.8e-06  3.51e-01   -1.814982218e+03  -1.814987782e+03  6.3e-08  0.01  
21  8.3e-07  3.8e-07  5.6e-07  7.11e-01   -1.814757852e+03  -1.814759391e+03  1.9e-08  0.01  
22  3.6e-07  1.7e-07  2.6e-07  3.25e-01   -1.814661212e+03  -1.814661145e+03  8.4e-09  0.01  
23  1.1e-07  5.1e-08  5.2e-08  6.92e-01   -1.814542600e+03  -1.814542341e+03  2.6e-09  0.01  
24  4.7e-08  2.2e-08  2.3e-08  3.06e-01   -1.814492608e+03  -1.814491846e+03  1.1e-09  0.01  
25  1